In [14]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import model_selection, preprocessing
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from xgboost.sklearn import XGBRegressor

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [19]:
df_train = pd.read_csv('clean_train.csv')
df_test = pd.read_csv('clean_test.csv')
id_test = df_test.ID
id_train = df_train.ID
df_train.shape, df_test.shape

((4209, 312), (4209, 311))

In [20]:
y_train = df_train["y"]
x_train = df_train.drop(["y"], axis=1)
x_test = df_test#.drop(["ID"], axis=1)

print x_train.shape, x_test.shape, y_train.shape

for c in x_train.columns:
    if x_train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train[c].values)) 
        x_train[c] = lbl.transform(list(x_train[c].values))
        #x_train.drop(c,axis=1,inplace=True)
        
for c in x_test.columns:
    if x_test[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_test[c].values)) 
        x_test[c] = lbl.transform(list(x_test[c].values))
        #x_test.drop(c,axis=1,inplace=True) 

print x_train.shape, x_test.shape

(4209, 311) (4209, 311) (4209,)
(4209, 311) (4209, 311)


In [4]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=2020)

In [77]:
params = {  
    "n_estimators": range(80, 121, 10),
    "max_depth": range(1,2),
    "learning_rate": np.arange(0.01,0.3, 0.01),
    "colsample_bytree": np.arange(0.3, 0.8, 0.05),
    "subsample": np.arange(0.3, 0.8, 0.05),
    "gamma": range(0, 10)
#     'reg_alpha': from_zero_positive,
#     "min_child_weight": from_zero_positive,
}
iters = 100

xgbreg = XGBRegressor()

In [78]:
gs = RandomizedSearchCV(xgbreg, params, n_iter=iters, n_jobs=5, scoring='r2')#rmsle_scorer)
gs.fit(x_train, y_train)

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
          fit_params={}, iid=True, n_iter=100, n_jobs=5,
          param_distributions={'colsample_bytree': array([ 0.3 ,  0.35,  0.4 ,  0.45,  0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75]), 'learning_rate': array([ 0.01,  0.02,  0.03,  0.04,  0.05,  0.06,  0.07,  0.08,  0.09,
        0.1 ,  0.11,  0.12,  0.13,  0.14,  0.15,  0.16,  0.17,  0.18,
        0.19,  0.2 ,  0....5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75]), 'max_depth': [1], 'gamma': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring='r2', verbose=0)

In [79]:
# gs.cv_results_

In [80]:
df = pd.DataFrame(gs.cv_results_)

In [104]:
df['diff'] = abs(df['mean_test_score'] - df['mean_train_score'])
df['mean'] = df[['mean_test_score', 'mean_train_score']].mean(axis=1)
priority_cols = ['diff', 'mean', 'mean_train_score','mean_test_score']
df = df[priority_cols + list(df.columns.drop(priority_cols))]

subset = df[(df['mean_test_score'] > 0.55) & (df['diff'] < 0.02)]
# df.sort(columns=['diff'], inplace=True, ascending=True)
subset.sort(columns=['mean_test_score'], inplace=True, ascending=False)
subset

/home/tseidakhmetov/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/tseidakhmetov/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:3304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  na_position=na_position)


,diff,mean,mean_train_score,mean_test_score,mean_fit_time,mean_score_time,param_colsample_bytree,param_gamma,param_learning_rate,param_max_depth,param_n_estimators,param_subsample,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
48,0.016862,0.562813,0.571244,0.554382,0.652909,0.044243,0.6,7,0.15,1,90,0.7,"{u'colsample_bytree': 0.6, u'learning_rate': 0...",8,0.572326,0.557883,0.594310,0.552319,0.496493,0.603529,0.078160,0.004064,0.041897,0.022942
53,0.017992,0.563227,0.572223,0.554230,0.686188,0.048960,0.6,8,0.12,1,110,0.45,"{u'colsample_bytree': 0.6, u'learning_rate': 0...",10,0.572666,0.557979,0.594186,0.553320,0.495823,0.605370,0.040531,0.001609,0.042216,0.023516
5,0.015301,0.561274,0.568924,0.553623,0.701105,0.051632,0.45,0,0.13,1,100,0.75,"{u'colsample_bytree': 0.45, u'learning_rate': ...",15,0.569890,0.555224,0.595424,0.550399,0.495542,0.601149,0.080903,0.001847,0.042363,0.022872
78,0.017620,0.561438,0.570248,0.552628,0.688826,0.047011,0.45,9,0.13,1,100,0.55,"{u'colsample_bytree': 0.45, u'learning_rate': ...",31,0.567891,0.556622,0.594645,0.550836,0.495335,0.603287,0.096899,0.000747,0.041951,0.023481
27,0.018624,0.561929,0.571241,0.552618,0.883304,0.056795,0.65,5,0.12,1,110,0.65,"{u'colsample_bytree': 0.65, u'learning_rate': ...",32,0.566566,0.557235,0.595811,0.552856,0.495464,0.603631,0.133734,0.009473,0.042132,0.022973
88,0.019429,0.561400,0.571114,0.551685,0.656423,0.046997,0.7,1,0.17,1,80,0.75,"{u'colsample_bytree': 0.7, u'learning_rate': 0...",38,0.562011,0.557526,0.596140,0.552088,0.496896,0.603728,0.074970,0.000837,0.041164,0.023168
37,0.012027,0.557677,0.563691,0.551664,0.611699,0.046192,0.4,8,0.12,1,90,0.65,"{u'colsample_bytree': 0.4, u'learning_rate': 0...",39,0.565235,0.549363,0.593962,0.545551,0.495782,0.596159,0.013237,0.006855,0.041210,0.023011
91,0.017309,0.560260,0.568914,0.551606,0.682702,0.055784,0.3,7,0.14,1,100,0.7,"{u'colsample_bytree': 0.3, u'learning_rate': 0...",41,0.563073,0.554090,0.594169,0.551033,0.497565,0.601619,0.070128,0.008318,0.040259,0.023159
87,0.018168,0.560369,0.569453,0.551286,0.701383,0.051615,0.7,0,0.12,1,100,0.55,"{u'colsample_bytree': 0.7, u'learning_rate': 0...",44,0.564253,0.555320,0.594228,0.551103,0.495364,0.601936,0.068835,0.011225,0.041385,0.023034
26,0.018168,0.560369,0.569453,0.551286,0.740707,0.056304,0.7,8,0.12,1,100,0.55,"{u'colsample_bytree': 0.7, u'learning_rate': 0...",44,0.564253,0.555320,0.594228,0.551103,0.495364,0.601936,0.092743,0.006593,0.041385,0.023034


In [54]:
df.ix[97].params

{'colsample_bytree': 0.79999999999999982,
 'gamma': 8,
 'learning_rate': 0.050000000000000003,
 'max_depth': 2,
 'subsample': 0.34999999999999998}

In [7]:
gs.best_score_

0.57136525821272444

In [5]:
# model = gs.best_estimator_

# model = XGBRegressor(colsample_bytree = 0.6,
#                      gamma =  7,
#                      learning_rate = 0.15,
#                      max_depth = 1,
#                      n_estimators = 90,
#                      subsample = 0.7)
model = XGBRegressor(colsample_bytree = 0.4,
                     gamma =  1,
                     learning_rate = 0.09,
                     max_depth = 2,
                     n_estimators = 100,
                     nthread = 3,
                     seed=2017,
                     subsample = 0.9)
model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.4,
       gamma=1, learning_rate=0.09, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=100, nthread=3,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=2017, silent=True, subsample=0.9)

In [6]:
cv_scores = cross_val_score(model, x_train, y_train, scoring='r2', cv=5)
print cv_scores
print np.mean(cv_scores)

[ 0.60139482  0.57479816  0.60967837  0.46828866  0.61033237]
0.572898477659


In [7]:
y_pred_tr = model.predict(x_train)
print 'Train:', r2_score(y_train, y_pred_tr)

y_pred_val = model.predict(x_valid)
print 'Val:', r2_score(y_valid, y_pred_val)
# Train: ('r2', 0.60473081638029402)
# Val: ('r2', 0.56729882111087759)

# Train: ('r2', 0.59019025799407598)
# Val: ('r2', 0.56887638635435567)

# 0.6
# Train: ('r2', 0.58691791141382998)
# Val: ('r2', 0.5681459527865238)

Train: 0.595457979099
Val: 0.564799912711


In [23]:
y_predict = model.predict(x_train)
output = pd.DataFrame({'ID': id_train, 'y_actual': y_train, 'y': y_predict})
print output.shape
output.head()

(4209, 3)


,ID,y,y_actual
0,0,112.488533,130.81
1,6,96.286659,88.53
2,7,82.812004,76.26
3,9,79.370132,80.62
4,13,81.264946,78.02


In [24]:
output.to_csv('train_preds/xgb_sklearn100_ix48.csv', index=False)

In [10]:
y_predict = model.predict(x_test)
output = pd.DataFrame({'ID': id_test, 'y': y_predict})
output.head()

,ID,y
0,1,86.574738
1,2,103.893013
2,3,83.002457
3,4,79.078148
4,5,117.709808


In [11]:
output.to_csv('submissions/xgb_sklearn100_ix48.csv', index=False)

In [64]:
output.shape

(4209, 2)

In [65]:
sample = pd.read_csv('sample_submission.csv')
sample.shape

(4209, 2)

In [66]:
sample.head()

,ID,y
0,1,100.669318
1,2,100.669318
2,3,100.669318
3,4,100.669318
4,5,100.669318


In [12]:
model.feature_importances_

AttributeError: 'XGBRegressor' object has no attribute 'feature_importances_'